# Azure OpenAI

In this quickstart you will create a simple Llama Index App and learn how to log it and get feedback on an LLM response using both an embedding and chat completion model from Azure OpenAI.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/trulens_eval/examples/expositional/models/azure_openai.ipynb)

## Setup

### Install dependencies
Let's install some of the dependencies for this notebook if we don't have them already

In [ ]:
#! pip install trulens-eval==0.21.0 llama_index==0.9.13 langchain==0.0.346 html2text==2020.1.16

### Add API keys
For this quickstart, you will need a larger set of information from Azure OpenAI compared to typical OpenAI usage. These can be retrieved from https://oai.azure.com/ . Deployment name below is also found on the oai azure page.

In [ ]:
# Check your https://oai.azure.com dashboard to retrieve params:

import os
os.environ["AZURE_OPENAI_API_KEY"] = "..." # azure
os.environ["AZURE_OPENAI_ENDPOINT"] = "..." # azure
os.environ["OPENAI_API_VERSION"] = "2023-07-01-preview" # may need updating
os.environ["OPENAI_API_TYPE"] = "azure"

### Import from TruLens

In [ ]:
# Imports main tools:
from trulens_eval import TruLlama, Feedback, Tru
tru = Tru()
tru.reset_database()

### Create Simple LLM Application

This example uses LlamaIndex which internally uses an OpenAI LLM.

In [ ]:
import os
from llama_index.llms import AzureOpenAI
from llama_index.embeddings import AzureOpenAIEmbedding
from llama_index import VectorStoreIndex, ServiceContext
from llama_index.readers import SimpleWebPageReader
from llama_index import set_global_service_context
import logging
import sys

# get model from Azure
llm = AzureOpenAI(
    model="gpt-35-turbo",
    deployment_name="truera-gpt-35-turbo",
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_version=os.environ["OPENAI_API_VERSION"],
)

# You need to deploy your own embedding model as well as your own chat completion model
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="truera-text-embedding-ada-002",
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_version=os.environ["OPENAI_API_VERSION"],
)

documents = SimpleWebPageReader(html_to_text=True).load_data(
    ["http://paulgraham.com/worked.html"]
)

service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
)

set_global_service_context(service_context)

index = VectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine()

### Send your first request

In [ ]:
query = "What is most interesting about this essay?"
answer = query_engine.query(query)

print(answer.get_formatted_sources())
print("query was:", query)
print("answer was:", answer)

## Initialize Feedback Function(s)

In [ ]:
from trulens_eval.feedback.provider import AzureOpenAI
from trulens_eval.feedback import Groundedness

import numpy as np
# Initialize AzureOpenAI-based feedback function collection class:
azopenai = AzureOpenAI(
    deployment_name="truera-gpt-35-turbo")

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(azopenai.relevance, name = "Answer Relevance").on_input_output()

# Question/statement relevance between question and each context chunk.
f_qs_relevance = Feedback(azopenai.qs_relevance_with_cot_reasons, name = "Context Relevance").on_input().on(
    TruLlama.select_source_nodes().node.text
).aggregate(np.mean)

# groundedness of output on the context
groundedness = Groundedness(groundedness_provider=azopenai)
f_groundedness = (Feedback(groundedness.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(TruLlama.select_source_nodes().node.text.collect())
    .on_output()
    .aggregate(groundedness.grounded_statements_aggregator)
)

## Custom functions can also use the Azure provider

In [ ]:
from typing import Tuple, Dict
from trulens_eval.feedback import prompts

class Custom_AzureOpenAI(AzureOpenAI):
    def qs_relevance_with_cot_reasons_extreme(self, question: str, statement: str) -> Tuple[float, Dict]:
        """
        Uses chat completion model. A function that completes a
        template to check the relevance of the statement to the question.
        Scoring guidelines for scores 5-8 are removed to push the LLM to more extreme scores.
        Also uses chain of thought methodology and emits the reasons.

        Args:
            question (str): A question being asked. 
            statement (str): A statement to the question.

        Returns:
            float: A value between 0 and 1. 0 being "not relevant" and 1 being "relevant".
        """

        system_prompt = str.format(prompts.QS_RELEVANCE, question = question, statement = statement)

        # remove scoring guidelines around middle scores
        system_prompt = system_prompt.replace(
    "- STATEMENT that is RELEVANT to most of the QUESTION should get a score of 5, 6, 7 or 8. Higher score indicates more RELEVANCE.\n\n", "")
        
        system_prompt = system_prompt.replace(
            "RELEVANCE:", prompts.COT_REASONS_TEMPLATE
        )

        return self._extract_score_and_reasons_from_response(system_prompt)
    
custom_azopenai = Custom_AzureOpenAI(deployment_name="truera-gpt-35-turbo")
    
# Question/statement relevance between question and each context chunk.
f_qs_relevance_extreme = (
    Feedback(custom_azopenai.qs_relevance_with_cot_reasons_extreme, name = "Context Relevance - Extreme")
    .on_input()
    .on(TruLlama.select_source_nodes().node.text)
    .aggregate(np.mean)
)

## Instrument chain for logging with TruLens

In [ ]:
tru_query_engine_recorder = TruLlama(query_engine,
    app_id='LlamaIndex_App1',
    feedbacks=[f_groundedness, f_qa_relevance, f_qs_relevance, f_qs_relevance_extreme])

In [ ]:
query = "What is most interesting about this essay?"
with tru_query_engine_recorder as recording:
    answer = query_engine.query(query)
    print(answer.get_formatted_sources())
    print("query was:", query)
    print("answer was:", answer)

## Explore in a Dashboard

In [ ]:
tru.run_dashboard(force=True) # open a local streamlit app to explore

# tru.stop_dashboard() # stop if needed

Alternatively, you can run `trulens-eval` from a command line in the same folder to start the dashboard.

## Or view results directly in your notebook

In [ ]:
tru.get_records_and_feedback(app_ids=[])[0] # pass an empty list of app_ids to get all